# Optimizer Comparison Experiments
## CSE 493S Project - Systematic Comparison of Optimization Algorithms

**Team Members:** Jinghao Liu, Xuan Zhang, Yuzheng Zhang

This notebook contains the code to run optimizer comparison experiments on CIFAR-10 and CIFAR-100.

**Optimizers tested:**
- SGD with Momentum
- Adam
- AdamW
- RAdam
- Lion

## Setup for Google Colab

In [ ]:
# Check if running on Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running on Google Colab")
except:
    IN_COLAB = False
    print("✓ Running locally")

In [ ]:
# Setup for Colab
if IN_COLAB:
    # Clone repository (replace with your repo URL)
    !git clone https://github.com/JasonHistoria/optimizer_benchmark.git
    %cd optimizer_benchmark
    
    # Install dependencies
    !pip install -q lion-pytorch tqdm pyyaml
    
    print("✓ Setup complete")
else:
    import sys
    sys.path.insert(0, 'src')
    print("✓ Using local setup")

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU detected. Training will be slow on CPU.")

## Import Libraries

In [ ]:
import torch
import torch.nn as nn
import sys
sys.path.insert(0, 'src')

from models import get_model, count_parameters
from data import get_dataloader
from optimizers import get_optimizer, get_scheduler, get_default_config
from utils import MetricsLogger, set_seed, AverageMeter, accuracy

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from IPython.display import clear_output
import json
import os

sns.set_style('whitegrid')
print("✓ All imports successful")

## Experiment Configuration

In [ ]:
# Experiment settings
CONFIG = {
    'dataset': 'cifar10',        # 'cifar10' or 'cifar100'
    'model': 'resnet18',         # 'resnet18', 'resnet34', or 'resnet50'
    'optimizer': 'adam',         # 'sgd', 'adam', 'adamw', 'radam', or 'lion'
    'epochs': 200,               # Number of training epochs
    'batch_size': 128,           # Batch size
    'seed': 42,                  # Random seed
    'lr': None,                  # Learning rate (None = use default)
    'weight_decay': None,        # Weight decay (None = use default)
    'scheduler': 'cosine',       # 'none', 'cosine', 'step', or 'multistep'
    'augment': True,             # Use data augmentation
    'workers': 4,                # Data loading workers
    
    # For hypothesis testing
    'label_noise': 0.0,          # Fraction of noisy labels (H3)
    'data_fraction': 1.0,        # Fraction of training data
}

# Use default hyperparameters if not specified
if CONFIG['lr'] is None or CONFIG['weight_decay'] is None:
    default_config = get_default_config(CONFIG['optimizer'])
    if CONFIG['lr'] is None:
        CONFIG['lr'] = default_config['lr']
    if CONFIG['weight_decay'] is None:
        CONFIG['weight_decay'] = default_config['weight_decay']

print("Experiment Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## Training Functions

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    model.train()
    losses = AverageMeter()
    top1 = AverageMeter()
    
    pbar = tqdm(train_loader, desc=f'Epoch {epoch:3d} [Train]', leave=False)
    for inputs, targets in pbar:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = accuracy(outputs, targets)[0]
        losses.update(loss.item(), inputs.size(0))
        top1.update(acc.item(), inputs.size(0))
        pbar.set_postfix({'loss': f'{losses.avg:.4f}', 'acc': f'{top1.avg:.2f}%'})
    
    return losses.avg, top1.avg

def validate(model, test_loader, criterion, device):
    model.eval()
    losses = AverageMeter()
    top1 = AverageMeter()
    
    with torch.no_grad():
        for inputs, targets in tqdm(test_loader, desc='[Val]', leave=False):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            acc = accuracy(outputs, targets)[0]
            losses.update(loss.item(), inputs.size(0))
            top1.update(acc.item(), inputs.size(0))
    
    return losses.avg, top1.avg

print("✓ Training functions defined")

## Load Data and Model

In [ ]:
# Set random seed
set_seed(CONFIG['seed'])
# Set device (support CUDA, MPS, and CPU)
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# Load data
print(f"\nLoading {CONFIG['dataset'].upper()} dataset...")
train_loader, test_loader, num_classes = get_dataloader(
    dataset=CONFIG['dataset'],
    batch_size=CONFIG['batch_size'],
    augment=CONFIG['augment'],
    num_workers=CONFIG['workers'],
    data_fraction=CONFIG['data_fraction'],
    label_noise=CONFIG['label_noise']
)
print(f"  Train batches: {len(train_loader)}")
print(f"  Test batches: {len(test_loader)}")

# Create model
print(f"\nCreating {CONFIG['model']} model...")
model = get_model(CONFIG['model'], num_classes=num_classes).to(device)
print(f"  Parameters: {count_parameters(model):,}")

# Create optimizer
print(f"\nCreating {CONFIG['optimizer'].upper()} optimizer...")
optimizer = get_optimizer(
    CONFIG['optimizer'], model.parameters(),
    lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay']
)
print(f"  LR: {CONFIG['lr']}, WD: {CONFIG['weight_decay']}")

# Create scheduler
scheduler = get_scheduler(optimizer, CONFIG['scheduler'], CONFIG['epochs'])
criterion = nn.CrossEntropyLoss()
print("\n✓ Setup complete!")

## Training Loop

In [ ]:
history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': [], 'learning_rate': []}
best_acc = 0.0

print(f"Training for {CONFIG['epochs']} epochs...")
print("=" * 70)

for epoch in range(1, CONFIG['epochs'] + 1):
    current_lr = optimizer.param_groups[0]['lr']
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, epoch)
    test_loss, test_acc = validate(model, test_loader, criterion, device)
    
    if scheduler:
        scheduler.step()
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['test_loss'].append(test_loss)
    history['test_acc'].append(test_acc)
    history['learning_rate'].append(current_lr)
    
    print(f"Epoch {epoch:3d}/{CONFIG['epochs']} | "
          f"Train: {train_loss:.4f} / {train_acc:6.2f}% | "
          f"Test: {test_loss:.4f} / {test_acc:6.2f}% | LR: {current_lr:.6f}")
    
    if test_acc > best_acc:
        best_acc = test_acc
        print(f"  ✓ New best: {best_acc:.2f}%")

print("\n" + "=" * 70)
print(f"Training complete! Best: {best_acc:.2f}% | Final: {test_acc:.2f}%")

## Visualize Results

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
epochs_range = range(1, len(history['train_loss']) + 1)

# Loss curves
axes[0, 0].plot(epochs_range, history['train_loss'], linewidth=2, label='Train')
axes[0, 0].plot(epochs_range, history['test_loss'], linewidth=2, label='Test')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Accuracy curves
axes[0, 1].plot(epochs_range, history['train_acc'], linewidth=2, label='Train')
axes[0, 1].plot(epochs_range, history['test_acc'], linewidth=2, label='Test')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy (%)')
axes[0, 1].set_title('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Test loss detail
axes[1, 0].plot(epochs_range, history['test_loss'], linewidth=2, color='red')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Test Loss')
axes[1, 0].set_title('Test Loss (Detail)')
axes[1, 0].grid(True, alpha=0.3)

# Learning rate
axes[1, 1].plot(epochs_range, history['learning_rate'], linewidth=2, color='green')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Learning Rate')
axes[1, 1].set_title('LR Schedule')
axes[1, 1].set_yscale('log')
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle(f"{CONFIG['optimizer'].upper()} on {CONFIG['dataset'].upper()} - Best: {best_acc:.2f}%", 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

if IN_COLAB:
    plt.savefig(f"{CONFIG['optimizer']}_{CONFIG['dataset']}_results.png", dpi=300, bbox_inches='tight')
    print(f"\n✓ Plot saved")

## Save Results

In [ ]:
os.makedirs('results', exist_ok=True)
exp_name = f"{CONFIG['dataset']}_{CONFIG['optimizer']}_seed{CONFIG['seed']}"
results_file = f'results/{exp_name}_metrics.json'

with open(results_file, 'w') as f:
    json.dump({
        'config': CONFIG,
        'history': history,
        'best_accuracy': float(best_acc),
        'final_accuracy': float(test_acc)
    }, f, indent=2)

print(f"✓ Results saved to {results_file}")

# Save model
checkpoint_file = f'results/{exp_name}_model.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'best_accuracy': best_acc,
    'config': CONFIG
}, checkpoint_file)
print(f"✓ Model saved to {checkpoint_file}")

if IN_COLAB:
    from google.colab import files
    files.download(results_file)
    print("✓ Results downloaded")